In [1]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 78.9 MB/s eta 0:00:00


In [9]:
import numpy as np
import json
import requests
from io import BytesIO
import pickle
import pandas as pd
import time
import implicit
import scipy

# Función para calcular métricas

In [15]:
# funcion de una issue de implicit cuando no funcionan las metricas de implicit
# https://github.com/benfred/implicit/issues/726
def ranking_metrics_at_k(model, train_user_items, test_user_items, K=10, show_progress=True):
    """
    Calculates ranking metrics (Precision@K, MAP@K, NDCG@K, AUC) for a trained model.

    Parameters:
        model : Trained ALS model (or other Implicit model).
        train_user_items : csr_matrix
            User-item interaction matrix used for training.
        test_user_items : csr_matrix
            User-item interaction matrix for evaluation.
        K : int
            Number of items to evaluate.
        show_progress : bool
            Show a progress bar during evaluation.

    Returns:
        dict : Dictionary with precision, MAP, NDCG, and AUC scores.
    """

    # Ensure matrices are in CSR format
    train_user_items = train_user_items.tocsr()
    test_user_items = test_user_items.tocsr()

    num_users, num_items = test_user_items.shape
    relevant = 0
    total_precision_div = 0
    total_map = 0
    total_ndcg = 0
    total_auc = 0
    total_users = 0

    # Compute cumulative gain for NDCG normalization
    cg = 1.0 / np.log2(np.arange(2, K + 2))  # Discount factor
    cg_sum = np.cumsum(cg)  # Ideal DCG normalization

    # Get users with at least one item in the test set
    users_with_test_data = np.where(np.diff(test_user_items.indptr) > 0)[0]

    # Progress bar
    #progress = tqdm.tqdm(total=len(users_with_test_data), disable=not show_progress)

    batch_size = 1000
    start_idx = 0

    while start_idx < len(users_with_test_data):
        batch_users = users_with_test_data[start_idx:start_idx + batch_size]
        recommended_items, _ = model.recommend(batch_users, train_user_items[batch_users], N=K)
        start_idx += batch_size

        for user_idx, user_id in enumerate(batch_users):
            test_items = set(test_user_items.indices[test_user_items.indptr[user_id]:test_user_items.indptr[user_id + 1]])

            if not test_items:
                continue  # Skip users without test data

            num_relevant = len(test_items)
            total_precision_div += min(K, num_relevant)

            ap = 0
            hit_count = 0
            auc = 0
            idcg = cg_sum[min(K, num_relevant) - 1]  # Ideal Discounted Cumulative Gain (IDCG)
            num_negative = num_items - num_relevant

            for rank, item in enumerate(recommended_items[user_idx]):
                if item in test_items:
                    relevant += 1
                    hit_count += 1
                    ap += hit_count / (rank + 1)
                    total_ndcg += cg[rank] / idcg
                else:
                    auc += hit_count  # Accumulate hits for AUC calculation

            auc += ((hit_count + num_relevant) / 2.0) * (num_negative - (K - hit_count))
            total_map += ap / min(K, num_relevant)
            total_auc += auc / (num_relevant * num_negative)
            total_users += 1

        #progress.update(len(batch_users))

    #progress.close()

    # Compute final metrics
    precision = relevant / total_precision_div if total_precision_div > 0 else 0
    mean_ap = total_map / total_users if total_users > 0 else 0
    mean_ndcg = total_ndcg / total_users if total_users > 0 else 0
    mean_auc = total_auc / total_users if total_users > 0 else 0

    return {
        "precision": precision,
        "map": mean_ap,
        "ndcg": mean_ndcg,
        "auc": mean_auc
    }

# funcion para calcular metricas dadas las recomendaciones ya hechas
def recs_ranking_metrics_at_k(recommendations, test_user_items, K=10):

    # Ensure matrices are in CSR format
    #train_user_items = train_user_items.tocsr()
    test_user_items = test_user_items.tocsr()

    num_users, num_items = test_user_items.shape
    relevant = 0
    total_precision_div = 0
    total_map = 0
    total_ndcg = 0
    total_auc = 0
    total_users = 0

    # Compute cumulative gain for NDCG normalization
    cg = 1.0 / np.log2(np.arange(2, K + 2))  # Discount factor
    cg_sum = np.cumsum(cg)  # Ideal DCG normalization

    # Get users with at least one item in the test set
    users_with_test_data = np.where(np.diff(test_user_items.indptr) > 0)[0]

    # Progress bar
    #progress = tqdm.tqdm(total=len(users_with_test_data), disable=not show_progress)

    batch_size = 1000
    start_idx = 0

    while start_idx < len(users_with_test_data):
        batch_users = users_with_test_data[start_idx:start_idx + batch_size]
        #recommended_items, _ = model.recommend(batch_users, train_user_items[batch_users], N=K)
        recommended_items = recommendations[start_idx:start_idx + batch_size]
        start_idx += batch_size

        for user_idx, user_id in enumerate(batch_users):
            test_items = set(test_user_items.indices[test_user_items.indptr[user_id]:test_user_items.indptr[user_id + 1]])

            if not test_items:
                continue  # Skip users without test data

            num_relevant = len(test_items)
            total_precision_div += min(K, num_relevant)

            ap = 0
            hit_count = 0
            auc = 0
            idcg = cg_sum[min(K, num_relevant) - 1]  # Ideal Discounted Cumulative Gain (IDCG)
            num_negative = num_items - num_relevant

            for rank, item in enumerate(recommended_items[user_idx]):
                if item in test_items:
                    relevant += 1
                    hit_count += 1
                    ap += hit_count / (rank + 1)
                    total_ndcg += cg[rank] / idcg
                else:
                    auc += hit_count  # Accumulate hits for AUC calculation

            auc += ((hit_count + num_relevant) / 2.0) * (num_negative - (K - hit_count))
            total_map += ap / min(K, num_relevant)
            total_auc += auc / (num_relevant * num_negative)
            total_users += 1

        #progress.update(len(batch_users))

    #progress.close()

    # Compute final metrics
    precision = relevant / total_precision_div if total_precision_div > 0 else 0
    mean_ap = total_map / total_users if total_users > 0 else 0
    mean_ndcg = total_ndcg / total_users if total_users > 0 else 0
    mean_auc = total_auc / total_users if total_users > 0 else 0

    return {
        "precision": precision,
        "map": mean_ap,
        "ndcg": mean_ndcg,
        "auc": mean_auc
    }

# Datos

## Descarga

In [3]:
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

--2025-05-09 21:33:23--  https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/4crrafugs93pzf3xshahz/goodreads_past_interactions.json?rlkey=w58hjav618jk0iyti1c4f85ad&dl=0 [following]
--2025-05-09 21:33:23--  https://www.dropbox.com/scl/fi/4crrafugs93pzf3xshahz/goodreads_past_interactions.json?rlkey=w58hjav618jk0iyti1c4f85ad&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc58e576bf7ff18238ca58990d1a.dl.dropboxusercontent.com/cd/0/inline/CpaPwtSW76uQE3urNpPBkZXyuqVWzO_mvmRK8ogtf4oFFChPx4jpU1F4HWBWyQNp5I0vNti9Jx6N9hYayTMhlS6AvDN5l2RCQu1hsCz1aS-dJvnEFT5HEhYPbAWAaBcgRjPPrD_2epJ2XoLKMvvNi_zA/file# [following]
--2025-05-09 21:33:24--  h

## Cargar datos como diccionarios

In [4]:
# diccionario con id del usuario y id de libros con los que ha interactuado en el pasado
with open('goodreads_past_interactions.json') as f:
    user_interactions = json.load(f)

# diccionario con id del usuario y id de libros para testear el modelo
with open('goodreads_test_interactions.json') as f:
    user_interactions_test = json.load(f)

## Convertir a matrices sparse para uso con Implicit

In [10]:
# convertimos diccionarios en matrices sparse (COO)
rows_train = []
cols_train = []
data_train = []

for user_id, book_ids in user_interactions.items():
    for book_id in book_ids:
        rows_train.append(int(user_id))
        cols_train.append(int(book_id))
        data_train.append(1)

max_user_id = max(rows_train)
max_book_id = max(cols_train)

rows_test = []
cols_test = []
data_test = []

for user_id, book_ids in user_interactions_test.items():
    for book_id in book_ids:
        rows_test.append(int(user_id))
        cols_test.append(int(book_id))
        data_test.append(1)

max_user_id = max([max_user_id] + rows_test)
max_book_id = max([max_book_id] + cols_test)

user_item_matrix_train = scipy.sparse.coo_matrix((data_train, (rows_train, cols_train)), shape=(max_user_id + 1, max_book_id + 1))
user_item_matrix_test = scipy.sparse.coo_matrix((data_test, (rows_test, cols_test)), shape=(max_user_id + 1, max_book_id + 1))

# convertimos a formato CSR
user_item_matrix_train = user_item_matrix_train.tocsr()
user_item_matrix_test = user_item_matrix_test.tocsr()

print('Train shape: ' , user_item_matrix_train.shape)
print('Test shape:' , user_item_matrix_test.shape)

# Modelos Baseline

## Implicit

In [11]:
model_ALS = implicit.als.AlternatingLeastSquares()
model_ALS.fit(user_item_matrix_train)

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

Tiempo de ejecución: 33 segundos

In [13]:
model_BPR = implicit.bpr.BayesianPersonalizedRanking()
model_BPR.fit(user_item_matrix_train)

  0%|          | 0/100 [00:00<?, ?it/s]

Tiempo de ejecución: 3 minutos y 43 segundos

In [14]:
K = 10
metrics_ALS = ranking_metrics_at_k(model_ALS, user_item_matrix_train, user_item_matrix_test, K=K)
metrics_BPR = ranking_metrics_at_k(model_BPR, user_item_matrix_train, user_item_matrix_test, K=K)
print("ALS", metrics_ALS)
print("BPR", metrics_BPR)

ALS {'precision': 0.202, 'map': 0.11772222222222219, 'ndcg': np.float64(0.23627664179514019), 'auc': 0.6006222222222221}
BPR {'precision': 0.096, 'map': 0.04282301587301585, 'ndcg': np.float64(0.1069683931214237), 'auc': 0.5475567567567566}


## Random

In [16]:
# recomendamos libros aleatorios para los usuarios del set de testeo
random_recs = []
books_id_set = set(range(max_book_id + 1))
train_books_id_set = set(np.nonzero(user_item_matrix_train.toarray())[1])
available_books_id_set = books_id_set - train_books_id_set
available_books_id = list(available_books_id_set)
for user in user_interactions_test:
    random_recs.append(np.random.choice(available_books_id, size=K, replace=False))

random_metrics = recs_ranking_metrics_at_k(random_recs, user_item_matrix_test, K=K)
print(random_metrics)

{'precision': 0.0, 'map': 0.0, 'ndcg': 0.0, 'auc': 0.4994994994994996}


## Item-Item

In [18]:
model_ii = implicit.nearest_neighbours.CosineRecommender()
model_ii.fit(user_item_matrix_train)

/usr/local/lib/python3.11/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.024477005004882812 seconds
  warnings.warn(


  0%|          | 0/10000 [00:00<?, ?it/s]

Tiempo de ejecución: 2 segundos

In [21]:
metrics_II = ranking_metrics_at_k(model_ii, user_item_matrix_train.astype(float), user_item_matrix_test.astype(float), K=K)
print("Item-Item", metrics_II)

Item-Item {'precision': 0.16, 'map': 0.08150436507936508, 'ndcg': np.float64(0.17380040471433392), 'auc': 0.5795891891891892}
